<a href="https://colab.research.google.com/github/rfdjalandoni/SimplePaths/blob/main/PGC_Network_Analysis_070522ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing necessary libraries. Be careful not to delete any lines
import numpy as np
import pandas as pd
import networkx as nx
import operator
import collections
import scipy.stats as ss
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Simple Paths Algorithm function; Run this code block

def find_simple_paths(start_node, end_node,start_col, end_col, path,save_path, cutoff_length,directed, res=[]):
    data = pd.read_excel(path)
    rows=len(data)
    src_nodes=np.asarray(data.iloc[0:rows,start_col-1]) # extract the nodes from database
    trg_nodes=np.asarray(data.iloc[0:rows,end_col-1])
    if directed==True:
        G=nx.DiGraph() # uncomment for directed network
        mode = 'directed'
    else:
        G=nx.Graph() # uncomment for undirected network
        mode = 'undirected'
    for i in range(len(src_nodes)):
        if src_nodes[i] in res or trg_nodes[i] in res :
            pass
        else:
            G.add_edge(src_nodes[i],trg_nodes[i] , weight=1)
    path_list=list(nx.all_simple_paths(G, start_node, end_node, cutoff=cutoff_length-1)) ## Run this to execute simple paths algorithm

    ### Saving only the paths with length=cutoff_length. Lengths lesser than cutoff_length are not saved.
    ### Here, the results are also converted as a dataframe which is read as a table
    ### Upon saving, headers are also add: Node1, Node2, etc.
    ### Here you can change the file name for the excel file of the results first before running the code block

    paths_of_interest=np.array([i for i in path_list if len(i)==cutoff_length])
    nodenum_as_col=paths_of_interest.T
    data_to_dict={'Node'+str(i+1):paths_of_interest.T[i] for i in range(len(paths_of_interest.T))}
    dict_to_df = pd.DataFrame(data_to_dict)

    ## this next line creates an excel file of the paths with length=cutoff_length

    dict_to_df.to_excel(save_path+start_node+"_to_"+end_node+"_l"+str(cutoff_length)+"simple_path"+mode+".xlsx")
    dict_to_df.to_csv(save_path+start_node+"_to_"+end_node+"_l"+str(cutoff_length)+"simple_path"+mode+".csv")

    ## naming format: (You can change this as long as you put .xlsx in the end) "(database)_(startnode)_to_(endnode)_n(cutoff_length)_res(restriction1)_(restriction2 and so on)_(directed or undirected).xlsx"

    ### We start with Node 1, and you calculate the Max Node for Node 2 (call this Max Node 2), 
    ### then we want to find the Max Node 3 connected to Max Node 2. 
    ## After that, we find the Max Node 4 connected to Max Node 3. We do this all the way to the last node. -Dr. Young
    ### This code block is step 3-8 from Yumi's emails
    ### No need to change anything here. You can run this code immediately.
    max_path_nodes=[]
    for i in range(len(nodenum_as_col)):
        if i==0 or i==len(nodenum_as_col)-1:
            curr_col=np.asarray(dict_to_df['Node'+str(i+1)])
            max_path_nodes.append(ss.mode(curr_col)[0])
        elif i==1:
            prev_col=np.asarray(dict_to_df['Node'+str(i)])
            curr_col=np.asarray(dict_to_df['Node'+str(i+1)])
            a=[i for i in ss.mode(curr_col)[0]]
            max_path_nodes.append(a)
        else:
            prev_col=np.asarray(dict_to_df['Node'+str(i)])
            curr_col=np.asarray(dict_to_df['Node'+str(i+1)])
            b=np.where(np.in1d(prev_col,a)==True)[0]
            c={}
            for m in range(len(b)):
                c[curr_col[b[m]]]=collections.Counter(curr_col)[curr_col[b[m]]]
            d=max(c.items(), key=operator.itemgetter(1))[0]
            a=[d]
            max_path_nodes.append(a)
    max_path_nodes=np.asarray(max_path_nodes).flatten()
    max_path_to_dict={'max_path_node'+str(i+1):max_path_nodes[i] for i in range(len(max_path_nodes))}
    max_path_to_df = pd.DataFrame(max_path_to_dict, index=[0])
    print("max_path is identified as:",max_path_nodes)

    
    ### saving path as excel file
    ## You can change the file name
    max_path_to_df.to_excel(save_path+start_node+"_to_"+end_node+"_l"+str(cutoff_length)+"max_path.xlsx")
    max_path_to_df.to_csv(save_path+start_node+"_to_"+end_node+"_l"+str(cutoff_length)+"max_path.csv")

## Check your data

In [ ]:
load_path = "/content/drive/MyDrive/PGC-Collab-2/Datasets/signor-database.xlsx"
save_path = "/content/drive/MyDrive/PGC-Collab-2/new-paths/"
data = pd.read_excel(load_path) # read_csv if opening a csv file
data.head(2)

,ENTITYA,TYPEA,IDA,DATABASEA,ENTITYB,TYPEB,IDB,DATABASEB,EFFECT,MECHANISM,...,MODASEQ,MODIFICATIONB,MODBSEQ,PMID,DIRECT,NOTES,ANNOTATOR,SENTENCE,SCORE,SIGNOR_ID
0,HTR1B,protein,P28222,UNIPROT,GNAI1,protein,P63096,UNIPROT,up-regulates activity,binding,...,NaN,NaN,NaN,31160049.0,YES,GPCR-Ga dataset,Luana,Here we systematically quantified ligand-induc...,0.1364,SIGNOR-256720
1,HTR1B,protein,P28222,UNIPROT,GNAI3,protein,P08754,UNIPROT,up-regulates activity,binding,...,NaN,NaN,NaN,31160049.0,YES,GPCR-Ga dataset,Luana,Here we systematically quantified ligand-induc...,0.1364,SIGNOR-256863


### Edit and run the block below

In [ ]:
 find_simple_paths(start_node = 'ERBB2',  # starting node
                   end_node = 'MYC',      # end node 
                   start_col = 1,         # column number for start node
                   end_col = 5,           # column number for end node
                   path = load_path,      # path of the data to be loaded (no need to edit anything here)
                   save_path = save_path, # path where to save data       (no need to edit anything here)
                   cutoff_length = 4,     # length of the path you would like to render
                   directed=True,         # if you want the network to be directed, If note let directed=False
                   res=[]                 # add restrictions to the paths
                   )

max_path is identified as: ['ERBB2' 'CDK1' 'RUNX1' 'MYC']
